<a href="https://colab.research.google.com/github/IamArmanNikkhah/InstagramBOT/blob/main/InstaAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instagram Bot

In [ ]:
!pip install Instaloader

In [ ]:
import instaloader
import os
import shutil

In [ ]:
class InstaBot:
  
  def __init__(self):
    self.loader = instaloader.Instaloader()

  def login(self, username, password):
    # Login or load session
    self.loader.login(username, password)          # (login)
  
  def getFollowers(self, username):
    # Obtain profile metadata
    Followers_list = []
    profile = instaloader.Profile.from_username(self.loader.context, username)
    for profile in profile.get_followers():
      Followers_list.append(profile.username)
    return Followers_list

  def getFollowees(self, username):
    Followees_list = []
    profile = instaloader.Profile.from_username(self.loader.context, username)
    for profile in profile.get_followees():
      Followees_list.append(profile.username)
    return Followees_list
  
  def downloadProfilePic(self, usernames_list):
    for username in usernames_list:
      self.loader.download_profile(username, profile_pic_only=True)

  def find_jpg(self, path="./"):
    """
    Find all jpg files in a directory
    """
    jpg_files = []
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(".jpg"):
                os.rename(os.path.join(root, file), os.path.join(root, root[2:] + '.jpg'))
                jpg_files.append(os.path.join(root, file))
    return jpg_files

  def copy_jpg(self, jpg_files, new_path):
    """
    Copy jpg files in a new folder
    """
    for jpg_file in jpg_files:
        shutil.copy(jpg_file, new_path)
  
  def putALLtogether(self):
    path = "./"
    new_path = "/content/jpg_files"
    jpg_files = self.find_jpg(path)
    self.copy_jpg(jpg_files, new_path)

In [ ]:
bot = InstaBot()
bot.login("nikkhah_arman","gJu%HqobZ$3G5D7^V98Th")

In [ ]:
bot.downloadProfilePic(["markmanson","pryw.k","s__o___nour"])
bot.find_jpg("./")
bot.putALLtogether()

# Gender Classification

In [ ]:
import torch
import torch.nn as nn
import torchvision
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
from torchvision.io import read_image
from torchvision import datasets, models, transforms
from google.colab.patches import cv2_imshow
from torch.utils.data import Dataset

In [ ]:
!wget https://postechackr-my.sharepoint.com/:u:/g/personal/dongbinna_postech_ac_kr/EVd9bFWzqztMrXRDdNnCHQkBsHaM4n5_1q1fue77vtQVtw?download=1 -O classification_model.pth
!wget --no-check-certificate \
    https://raw.githubusercontent.com/computationalcore/introduction-to-opencv/master/assets/test.jpg \
    -O test.jpg
!wget --no-check-certificate \
    https://raw.githubusercontent.com/computationalcore/introduction-to-opencv/master/assets/haarcascade_frontalface_default.xml \
    -O haarcascade_frontalface_default.xml
!wget --no-check-certificate \
    https://raw.githubusercontent.com/computationalcore/introduction-to-opencv/master/assets/haarcascade_smile.xml \
    -O haarcascade_smile.xml
!wget --no-check-certificate \
    https://raw.githubusercontent.com/computationalcore/introduction-to-opencv/master/assets/haarcascade_eye.xml \
    -O haarcascade_eye.xml
!wget --no-check-certificate \
    https://raw.githubusercontent.com/computationalcore/introduction-to-opencv/master/utils/common.py \
    -O common.py

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # device object

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, dir_path = "/content/jpg_files"):
        self.img_dir        =  dir_path
        self.transforms_val = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        self.detector    = faceDetector()
        self.images_list = self.makeListofImages()

    def __getitem__(self, index):
        # TODO
        # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
        img_path = self.images_list[index]
        img      = Image.open(img_path)
        # 2. Preprocess the data (e.g. torchvision.Transform).
        face  = self.detector.returnFaces(img_path)
        face_t = self.transforms_val(face)
        # 3. Return a data pair (e.g. image and label).
        if face is None:
          return face_t, "Woman"
        else:
          return face_t, "NG"
    
    def __len__(self):
        # You should change 0 to the total size of your dataset.
        return len(self.images_list) 

    def makeListofImages(self):
      """
      Find all jpg files in a directory
      """
      jpg_files = []
      for root, dirs, files in os.walk(self.img_dir):
        for file in files:
          if file.endswith(".jpg"):
            jpg_files.append(os.path.join(root, file))
      return jpg_files

# Then, you can just use prebuilt torch's data loader. 
custom_dataset = CustomDataset()
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                           batch_size=16, 
                                           shuffle=True)

In [ ]:
class faceDetector:
  def __init__(self):
    self.face_cascade = cv2.CascadeClassifier('/content/haarcascade_frontalface_default.xml')

  def crop_face(self, image, face_coordinates):
    x, y, w, h = face_coordinates
    return image[y:y+h, x:x+w]

  def detect_face(self, image):
    faces = self.face_cascade.detectMultiScale(image, scaleFactor=1.2, minNeighbors=5)
    if len(faces) == 0:
        return None
    return faces[0]

  def resize_image(self, image, size=(224, 224)):
    return cv2.resize(image, size)

  def detect_and_crop_face(self, image):
    face = self.detect_face(image)
    if face is not None:
        return self.resize_image(self.crop_face(image, face))
    return None

  def load_image(self, path):
    image = cv2.imread(path)
    gray  = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return gray

  def returnFaces(self, img_path):
    image = self.load_image(img_path)
    face = self.detect_and_crop_face(image)
    return face
